In [4]:
from random import shuffle
import glob, scipy, pickle, itertools, re
import numpy as np
from IPython.display import clear_output
from pandas import read_csv
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import scipy.io as sio
import mat73
from collections import Counter
from tqdm import tqdm
subj = 'RDAzimuth'
filename = 'datasets/'+subj+'_seq.pkl' # _resized
print('File to be created: ' + filename)

File to be created: datasets/RDAzimuth_seq.pkl


In [6]:
mdpath = '../../'+subj+'/*.mat'
addr = glob.glob(mdpath)
# addr = addr[:5000]
print('Num. Files: '+str(len(addr)))

Num. Files: 182


In [7]:
addr[0]

'../../RDAzimuth\\11010000_1603570411_Raw_0_1_001.mat'

In [11]:
slash_id = addr[0].rfind("\\")
class_id = slash_id+3
frame_id = addr[0].rfind("_")
print('class_id:',class_id)
print(addr[0][class_id:class_id+2])
print('frame_id:',frame_id)
print('unique:',addr[0][slash_id+1:frame_id])

class_id: 18
01
frame_id: 43
unique: 11010000_1603570411_Raw_0_1


In [12]:
fname_ls = np.unique([a[slash_id+1:frame_id] for a in addr])
print('Num. uniques:',len(fname_ls))
p = np.random.permutation(len(fname_ls))

Num. uniques: 3


In [13]:
num_sample = 100
fname_ls = fname_ls[p]
fname_ls = fname_ls[:num_sample]
print('Selected num. samples: ', len(fname_ls))

Selected num. samples:  3


In [14]:
# check how many frames in a uniq id
cnt = 0
uniq_id = -1
for i in addr:
    if fname_ls[uniq_id] in i:
        cnt += 1
cnt

67

In [15]:
# one sample
data = mat73.loadmat(addr[0])['crop_frame']
data.shape

(55, 256, 8)

In [16]:
dataset = []
labels = []
for i,v in tqdm(enumerate(fname_ls),position=0):
    
    indices = []
    for k,a in enumerate(addr):
        if v in a:
            indices.append(k)
    subarray = np.zeros((len(indices),data.shape[0],data.shape[1],data.shape[2]),dtype=complex)
    for j in range(len(indices)):
        subarray[j] = mat73.loadmat(addr[indices[j]])['crop_frame']
    labels.append(int(addr[indices[j]][class_id:class_id+2])-1)
    dataset.append(subarray)

3it [00:04,  1.56s/it]


In [17]:
dataset = np.array(dataset, dtype ='object')

In [18]:
print('Classes: ', np.unique(sorted(labels)))
Counter(labels)

Classes:  [0]


Counter({0: 3})

In [19]:
x_tr, x_ts, y_tr, y_ts = train_test_split(dataset,np.array(to_categorical(labels)),test_size=0.2,random_state=1)
print(x_tr.shape)
print(y_tr.shape)
print(x_ts.shape)
print(y_ts.shape)

(2,)
(2, 1)
(1,)
(1, 1)


In [21]:
data = [x_tr,x_ts,y_tr,y_ts]
filename = filename.replace('.pkl','_'+str(num_sample)+'.pkl')
with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(data, output, pickle.HIGHEST_PROTOCOL)
print(filename+' created.')

datasets/RDAzimuth_seq_100.pkl created.


In [34]:
x_tr[0].shape

(67, 55, 256, 8)